In [1]:
import pandas as pd

## Read dataframes

In [2]:
def read_dataframe_part(filename):
    df = pd.read_csv(filename)
    df = df[['commentBody']]
    return df

In [3]:
dataframes = []
for i in range(2):
    df_part = read_dataframe_part(f'data/{i}.csv')
    dataframes.append(df_part)

init_df = pd.concat(dataframes)
init_df.drop_duplicates(inplace=True)
init_df = init_df['commentBody'].to_numpy()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (14,15,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
init_df.size

433405

In [5]:
init_df = init_df[:100000]

## Tokenization

In [6]:
%pip install nltk

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import nltk
from nltk import word_tokenize
import re

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/leemurus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
def preprocess(text):
    return text.replace('’', "'").lower()

In [10]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [11]:
df = [0] * len(init_df)

for index, text in enumerate(init_df):
    new_text = preprocess(text)
    new_text = remove_urls(new_text)
    df[index] = word_tokenize(new_text)

In [12]:
init_df[0]

'For all you Americans out there --- still rejoicing over the majority win of  Republicans over the Legislature of this land.<br/>Beware.<br/>Just like you would have been, if  there were any other kind of majority.<br/>The Founding Fathers had something like this in mind wheh they formed our Great Nation.<br/>It\'s part of the natural \'Checks &amp; Balances\' system that keeps this country on an even keel.<br/>But this is now being threatened -- with the majority of Executive, Legislative and Judidical power all in the hands of one political party.<br/>See to it that you remember our U.S. Constitution, and our Bill of Rights.<br/>Remember that "We", are still "the People".<br/>America belongs to all of us.<br/>And God help us all.'

In [13]:
df[0]

['for',
 'all',
 'you',
 'americans',
 'out',
 'there',
 '--',
 '-',
 'still',
 'rejoicing',
 'over',
 'the',
 'majority',
 'win',
 'of',
 'republicans',
 'over',
 'the',
 'legislature',
 'of',
 'this',
 'land.',
 '<',
 'br/',
 '>',
 'beware.',
 '<',
 'br/',
 '>',
 'just',
 'like',
 'you',
 'would',
 'have',
 'been',
 ',',
 'if',
 'there',
 'were',
 'any',
 'other',
 'kind',
 'of',
 'majority.',
 '<',
 'br/',
 '>',
 'the',
 'founding',
 'fathers',
 'had',
 'something',
 'like',
 'this',
 'in',
 'mind',
 'wheh',
 'they',
 'formed',
 'our',
 'great',
 'nation.',
 '<',
 'br/',
 '>',
 'it',
 "'s",
 'part',
 'of',
 'the',
 'natural',
 "'checks",
 '&',
 'amp',
 ';',
 'balances',
 "'",
 'system',
 'that',
 'keeps',
 'this',
 'country',
 'on',
 'an',
 'even',
 'keel.',
 '<',
 'br/',
 '>',
 'but',
 'this',
 'is',
 'now',
 'being',
 'threatened',
 '--',
 'with',
 'the',
 'majority',
 'of',
 'executive',
 ',',
 'legislative',
 'and',
 'judidical',
 'power',
 'all',
 'in',
 'the',
 'hands',
 'of',

## Remove stop words

In [14]:
import nltk
import numpy as np
from nltk.corpus import stopwords
import string

In [15]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leemurus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def remove_stop_words(words):
    return [word for word in words if word not in stop_words]

In [17]:
for index, text in enumerate(df):
    df[index] = remove_stop_words(text)

## Normalization

In [18]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/leemurus/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [19]:
def remove_trash(words):
    clear_words = []
    
    for word in words:
        tag = nltk.pos_tag([word])[0][1]

        if re.match('^[a-zA-Z]+$', word): 
            clear_words.append(word)
        
    result = set()
    for word, tag in nltk.pos_tag(clear_words):
        if tag[0] == 'N':
            result.add(word)
    
    return result

In [20]:
clear_df = [0] * len(df)

for index, text in enumerate(df):
    clear_df[index] = remove_trash(text)

## Lemmatization

In [23]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/leemurus/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [24]:
wordnet_lemmatizer = WordNetLemmatizer()

In [25]:
def lemmatizate(words):
    result = set()
    
    for word in words:
        result.add(wordnet_lemmatizer.lemmatize(word))
    
    return result

In [32]:
for index, text in enumerate(clear_df):
    clear_df[index] = lemmatizate(text)

In [33]:
from collections import Counter
cnt = Counter()

for words in clear_df:
    cnt.update(words)

In [37]:
top = cnt.most_common()[:50]
top

[('trump', 26841),
 ('people', 20059),
 ('time', 15565),
 ('president', 13690),
 ('year', 13281),
 ('country', 10625),
 ('way', 9872),
 ('thing', 8904),
 ('world', 7710),
 ('state', 7394),
 ('election', 7285),
 ('government', 6753),
 ('day', 6747),
 ('care', 6316),
 ('man', 6169),
 ('life', 5780),
 ('health', 5576),
 ('fact', 5565),
 ('party', 5336),
 ('republican', 5209),
 ('job', 5184),
 ('nothing', 5120),
 ('news', 5080),
 ('obama', 5075),
 ('something', 4979),
 ('power', 4936),
 ('office', 4906),
 ('american', 4891),
 ('article', 4751),
 ('problem', 4594),
 ('tax', 4538),
 ('work', 4516),
 ('medium', 4506),
 ('woman', 4437),
 ('anything', 4351),
 ('democrat', 4224),
 ('money', 4221),
 ('anyone', 4186),
 ('clinton', 4131),
 ('policy', 4074),
 ('house', 4034),
 ('nation', 3998),
 ('part', 3933),
 ('word', 3913),
 ('war', 3902),
 ('family', 3874),
 ('business', 3862),
 ('need', 3862),
 ('issue', 3832),
 ('change', 3817)]

In [55]:
top_words = list(item[0] for item in top)

In [71]:
logs = []
for index1 in range(len(top_words)):
    for index2 in range(index1 + 1, len(top_words)):
        for words in clear_df:
            if top_words[index1] in words and top_words[index2] in words:
                pair =(top_words[index1], top_words[index2])
                logs.append(pair)

In [72]:
import csv
f = open('data/res.csv', 'w')
writer = csv.writer(f)
writer.writerows(logs)
f.close()